In [205]:
import argparse
import json
import logging
import os
import sys

import pandas as pd

from utils.utils import separate_dps, file_tqdm, separate_lrs
from utils.tree_utils import *
from dataset import *

In [206]:
data_dir = "../data"
journalist = "sallykohn"
out_dir = "../data/"
n_ctx = 2000
max_width = 16
max_depth = 10
td = True
local_relation = True

num_dps = 0
num_classes = 3

journal_sort = pd.read_csv(os.path.join(data_dir, f'{journalist}/{journalist}_conv_labels.csv'))
num_sequences = len(set(journal_sort['conversation_id']))

journal_batch = journal_sort[["type", "possibly_sensitive", "lang", "reply_settings",
                                "retweet_count", "reply_count", "like_count", "quote_count", "impression_count",
                                "mentions", "urls", "labels"]]
ids = list(set(journal_sort['conversation_id']))
id_pair = {}
for idx in ids:
    id_pair[idx] = create_conversation_list(journal_sort[journal_sort['conversation_id']==idx], idx)


In [233]:
alice_rel = []
with open(os.path.join(data_dir, f'aliceysu/aliceysu_local_path.txt'), "r") as f:
    for line in tqdm(f, total=get_number_of_lines(f)):
        alice_rel.append(json.loads(line.strip()))

100%|██████████| 419/419 [00:00<00:00, 7471.17it/s]


In [281]:
def separate_lrs(lrs, id_map, id_re, max_len):
    def reformat(lrs, left):  # [left,right)
        new_lrs = []
        for idx, lr in enumerate(lrs):
            # lr -> dict: {idx:[],idx:[]}
            temp_lr = dict()
            for key, val in lr.items():
                if left <= id_map[key] < left + max_len:
                    temp_lr[id_re[id_map[key] - left]] = val
            new_lrs.append(temp_lr)
        return new_lrs

    half_len = int(max_len / 2)
    if len(lrs) <= max_len:
        return [[reformat(lrs, 0), 0]]

    aug_asts = [[reformat(lrs[:max_len], 0), 0]]
    i = half_len
    while i < len(lrs) - max_len:
        aug_asts.append([reformat(lrs[i: i + max_len], i), half_len])
        i += half_len
    idx = max_len - (len(lrs) - (i + half_len))
    aug_asts.append([reformat(lrs[len(lrs) - max_len:], len(lrs) - max_len), idx])
    return aug_asts



In [265]:
def id_mapping(name_li):
    count = 0
    id_map = {}
    id_re = {}
    for item in name_li:
        if item not in id_map.keys():
            id_map[item] = count
            id_re[count] = item
            count += 1
    #id_list = [id_map[item] for item in name_li]
    return id_map, id_re

name_list = []
for item in node_list:
    name_list.append(item.name)

id_map, id_re = id_mapping(name_list)

In [195]:

if local_relation:
    print("Save Relation between Child and Father")
    with open(os.path.join(out_dir, f'{journalist}/{journalist}_local_path.txt'), "w") as fout:
        for k in id_pair.keys():
            tree_root = build_tree(id_pair[k][0])
            
            tree_root.create_local_relation()
            node_list = tree_root.dfs()
            name_list = []
            for item in node_list:
                name_list.append(item.name)

            id_map, id_re = id_mapping(name_list)
            local_relations = TreeNode.extract_data(node_list,f=lambda node: node.local_relation)
            lrs = separate_lrs(local_relations, id_map, id_re, n_ctx)

            """for lr, extended in lrs:
                if extended != 0:
                    break
                if len(lr) - extended > 1:
                    json.dump(lr, fp=fout)  # each line is the json of a list [dict,dict,...]
                    num_dps += 1
                    fout.write("\n")"""
            json.dump(local_relations, fp=fout)
            fout.write("\n")




Save Relation between Child and Father


In [108]:
# sallykohn
# journal_sort = journal_sort.drop(10).reset_index(drop=True)
# journal_sort.to_csv(os.path.join(data_dir, f'{journalist}/{journalist}_conv_labels.csv'))

In [196]:
if td:
    print("Save Child Paths and Father Paths")

num_dps = 0
with open(os.path.join(out_dir, f'{journalist}/{journalist}_global_path.txt'), "w") as fout:
    for k in id_pair.keys():
        tree_root = build_tree(id_pair[k][0])
        #print(tree_root.name)
        tree_root.create_global_relation()
        node_list = tree_root.dfs()
        
        root_paths = TreeNode.extract_data(node_list,f=lambda node: clamp_and_slice_ids(
                node.global_relation, max_width=-1, max_depth=-1))
        asts = separate_dps(root_paths, n_ctx)

        for lr, extended in asts:
            if extended != 0:
                break
            if len(lr) - extended > 1:
                json.dump(lr, fp=fout)  # each line is the json of a list [dict,dict,...]
                num_dps += 1
                fout.write("\n")

Save Child Paths and Father Paths


In [197]:
prob = pkl.load(open(os.path.join(data_dir, f'{journalist}/{journalist}_edgeprob.pkl'), 'rb'))
    

In [204]:
len(journal_sort)

55278